In [1]:
for i in range(3):
    from model import *
    from SeLUNet_model import *
    from ResUNet_model import *
    from data import *
    import os
    import tensorflow as tf
    from skimage.io import imread
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    
    def evaluation(pred, true, verbose=1):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        threshold = 0.5
        for i, j in zip(pred, true):
            for x, y in zip(i, j):
                if x >= threshold and y >= threshold:
                    tp += 1
                elif x < threshold and y < threshold:
                    tn += 1
                elif x >= threshold and y < threshold:
                    fp += 1
                elif x < threshold and y >= threshold:
                    fn += 1
        sen = tp/(tp+fn)
        spe = tn/(tn+fp)
        iou = tp/(fn+fp+tp)
        dice = tp/(2*tp+fn+fp)

        if verbose==1:
            print("============================")
            print("Dice score : {:0.4f}".format(dice))
            print("Sensitivity score : {:0.4f}".format(sen))
            print("Specificity score : {:0.4f}".format(spe))
            print("IoU score : {:0.4f}".format(iou))
            print("============================")
        return dice, sen, spe, iou
    
    print('============================Process %d/3============================' % (i+1))
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    # os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')
    myGene = trainGenerator(5,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
    model = SeLUNet()
    model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit_generator(myGene,steps_per_epoch=500,epochs=10,callbacks=[model_checkpoint])

    testGene = testGenerator("data/membrane/test")
    results = model.predict_generator(testGene,15,verbose=1)
    saveResult("data/membrane/results",results)

    true_path = glob.glob("data/membrane/test_mask/*.png")
    pred_path = glob.glob("data/membrane/results/*.tif")
    true_path.sort()
    pred_path.sort()
    true_imds = []
    pred_imds = []
    for i in range(len(true_path)):
        img1 = Image.open(true_path[i])
        img1 = np.array(img1)
        img1 = img1[:,:,:3][:,:,0]
        img1 = img1 / 255
        img1[img1>0] == 1

        img2 = Image.open(pred_path[i])
        img2 = np.array(img2)
        img2 = cv2.resize(img2,(150,150))
        img2[img2>0] == 1

        true_imds.append(img1)
        pred_imds.append(img2)
        
    dice = []
    sen = []
    spe = []
    IOU = []

    for i in range(len(pred_imds)):
        dices, sens, spes, ious = evaluation(pred_imds[i], true_imds[i], verbose=0)
        dice.append(dices)
        sen.append(sens)
        spe.append(spes)
        IOU.append(ious)
        Dice = np.mean(dice)
        Sen = np.mean(sen)
        Spe = np.mean(spe)
        IoU = np.mean(IOU)

    print("============================")
    print("Dice score : {:0.4f}".format(Dice))
    print("Sensitivity score : {:0.4f}".format(Sen))
    print("Specificity score : {:0.4f}".format(Spe))
    print("IoU : {:0.4f}".format(IoU))
    print("============================")

    

Using TensorFlow backend.


============================Process 1/3============================


E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\SeLUNet_model.py:58: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=conv10)


Epoch 1/10
Found 41 images belonging to 1 classes.
Found 41 images belonging to 1 classes.
500/500 [==============================] - 197s 394ms/step - loss: 0.1043 - accuracy: 0.9817

Epoch 00001: loss improved from inf to 0.10418, saving model to unet_membrane.hdf5
Epoch 2/10
500/500 [==============================] - 189s 378ms/step - loss: 0.0952 - accuracy: 0.9832

Epoch 00002: loss improved from 0.10418 to 0.09553, saving model to unet_membrane.hdf5
Epoch 3/10
500/500 [==============================] - 189s 378ms/step - loss: 0.0921 - accuracy: 0.9832

Epoch 00003: loss improved from 0.09553 to 0.09195, saving model to unet_membrane.hdf5
Epoch 4/10
500/500 [==============================] - 187s 374ms/step - loss: 0.0894 - accuracy: 0.9832

Epoch 00004: loss improved from 0.09195 to 0.08949, saving model to unet_membrane.hdf5
Epoch 5/10
500/500 [==============================] - 187s 375ms/step - loss: 0.0878 - accuracy: 0.9832

Epoch 00005: loss improved from 0.08949 to 0.08788,

E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\0.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\1.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\2.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\3.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\4.tif

Dice score : 0.0000
Sensitivity score : 0.0000
Specificity score : 1.0000
IoU : 0.0000
============================Process 2/3============================
Epoch 1/10
Found 41 images belonging to 1 classes.
Found 41 images belonging to 1 classes.
500/500 [==============================] - 190s 380ms/step - loss: 0.0707 - accuracy: 0.9818

Epoch 00001: loss improved from inf to 0.07049, saving model to unet_membrane.hdf5
Epoch 2/10
500/500 [==============================] - 191s 382ms/step - loss: 0.0487 - accuracy: 0.9844

Epoch 00002: loss improved from 0.07049 to 0.04891, saving model to unet_membrane.hdf5
Epoch 3/10
500/500 [==============================] - 192s 383ms/step - loss: 0.0351 - accuracy: 0.9877

Epoch 00003: loss improved from 0.04891 to 0.03525, saving model to unet_membrane.hdf5
Epoch 4/10
500/500 [==============================] - 192s 384ms/step - loss: 0.0286 - accuracy: 0.9895

Epoch 00004: loss improved from 0.03525 to 0.02846, saving model to unet_membrane.hdf5
E

In [2]:
for i in range(1):
    from model import *
    from SeLUNet_model import *
    from ResUNet_model import *
    from data import *
    import os
    import tensorflow as tf
    from skimage.io import imread
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    
    def evaluation(pred, true, verbose=1):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        threshold = 0.5
        for i, j in zip(pred, true):
            for x, y in zip(i, j):
                if x >= threshold and y >= threshold:
                    tp += 1
                elif x < threshold and y < threshold:
                    tn += 1
                elif x >= threshold and y < threshold:
                    fp += 1
                elif x < threshold and y >= threshold:
                    fn += 1
        sen = tp/(tp+fn)
        spe = tn/(tn+fp)
        iou = tp/(fn+fp+tp)
        dice = tp/(2*tp+fn+fp)

        if verbose==1:
            print("============================")
            print("Dice score : {:0.4f}".format(dice))
            print("Sensitivity score : {:0.4f}".format(sen))
            print("Specificity score : {:0.4f}".format(spe))
            print("IoU score : {:0.4f}".format(iou))
            print("============================")
        return dice, sen, spe, iou
    
    print('============================Process %d/3============================' % (i+1))
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    # os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')
    myGene = trainGenerator(5,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
    model = unet()
    model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit_generator(myGene,steps_per_epoch=500,epochs=10,callbacks=[model_checkpoint])

    testGene = testGenerator("data/membrane/test")
    results = model.predict_generator(testGene,15,verbose=1)
    saveResult("data/membrane/results",results)

    true_path = glob.glob("data/membrane/test_mask/*.png")
    pred_path = glob.glob("data/membrane/results/*.tif")
    true_path.sort()
    pred_path.sort()
    true_imds = []
    pred_imds = []
    for i in range(len(true_path)):
        img1 = Image.open(true_path[i])
        img1 = np.array(img1)
        img1 = img1[:,:,:3][:,:,0]
        img1 = img1 / 255
        img1[img1>0] == 1

        img2 = Image.open(pred_path[i])
        img2 = np.array(img2)
        img2 = cv2.resize(img2,(150,150))
        img2[img2>0] == 1

        true_imds.append(img1)
        pred_imds.append(img2)
        
    dice = []
    sen = []
    spe = []
    IOU = []

    for i in range(len(pred_imds)):
        dices, sens, spes, ious = evaluation(pred_imds[i], true_imds[i], verbose=0)
        dice.append(dices)
        sen.append(sens)
        spe.append(spes)
        IOU.append(ious)
        Dice = np.mean(dice)
        Sen = np.mean(sen)
        Spe = np.mean(spe)
        IoU = np.mean(IOU)

    print("============================")
    print("Dice score : {:0.4f}".format(Dice))
    print("Sensitivity score : {:0.4f}".format(Sen))
    print("Specificity score : {:0.4f}".format(Spe))
    print("IoU : {:0.4f}".format(IoU))
    print("============================")

============================Process 1/3============================


E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\model.py:54: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Epoch 1/10
Found 41 images belonging to 1 classes.
Found 41 images belonging to 1 classes.
500/500 [==============================] - 140s 279ms/step - loss: 0.0567 - accuracy: 0.9822

Epoch 00001: loss improved from inf to 0.05604, saving model to unet_membrane.hdf5
Epoch 2/10
500/500 [==============================] - 140s 280ms/step - loss: 0.0346 - accuracy: 0.9878

Epoch 00002: loss improved from 0.05604 to 0.03470, saving model to unet_membrane.hdf5
Epoch 3/10
500/500 [==============================] - 141s 281ms/step - loss: 0.0238 - accuracy: 0.9908

Epoch 00003: loss improved from 0.03470 to 0.02385, saving model to unet_membrane.hdf5
Epoch 4/10
500/500 [==============================] - 140s 281ms/step - loss: 0.0150 - accuracy: 0.9939

Epoch 00004: loss improved from 0.02385 to 0.01505, saving model to unet_membrane.hdf5
Epoch 5/10
500/500 [==============================] - 140s 280ms/step - loss: 0.0141 - accuracy: 0.9943

Epoch 00005: loss improved from 0.01505 to 0.01397,

In [3]:
for i in range(3):
    from model import *
    from SeLUNet_model import *
    from ResUNet_model import *
    from data import *
    import os
    import tensorflow as tf
    from skimage.io import imread
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    
    def evaluation(pred, true, verbose=1):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        threshold = 0.5
        for i, j in zip(pred, true):
            for x, y in zip(i, j):
                if x >= threshold and y >= threshold:
                    tp += 1
                elif x < threshold and y < threshold:
                    tn += 1
                elif x >= threshold and y < threshold:
                    fp += 1
                elif x < threshold and y >= threshold:
                    fn += 1
        sen = tp/(tp+fn)
        spe = tn/(tn+fp)
        iou = tp/(fn+fp+tp)
        dice = tp/(2*tp+fn+fp)

        if verbose==1:
            print("============================")
            print("Dice score : {:0.4f}".format(dice))
            print("Sensitivity score : {:0.4f}".format(sen))
            print("Specificity score : {:0.4f}".format(spe))
            print("IoU score : {:0.4f}".format(iou))
            print("============================")
        return dice, sen, spe, iou
    
    print('============================Process %d/3============================' % (i+1))
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    # os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')
    myGene = trainGenerator(5,'data/membrane/train_old','image','label',data_gen_args,save_to_dir = None)
    model = unet()
    model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit_generator(myGene,steps_per_epoch=500,epochs=10,callbacks=[model_checkpoint])

    testGene = testGenerator("data/membrane/test_old")
    results = model.predict_generator(testGene,15,verbose=1)
    saveResult("data/membrane/results",results)

    true_path = glob.glob("data/membrane/test_old_mask/*.bmp")
    pred_path = glob.glob("data/membrane/results/*.tif")
    true_path.sort()
    pred_path.sort()
    true_imds = []
    pred_imds = []
    for i in range(len(true_path)):
        img1 = Image.open(true_path[i])
        img1 = np.array(img1)
        img1 = img1[:,:,:3][:,:,0]
        img1 = img1 / 255
        img1[img1>0] == 1

        img2 = Image.open(pred_path[i])
        img2 = np.array(img2)
        img2 = cv2.resize(img2,(150,150))
        img2[img2>0] == 1

        true_imds.append(img1)
        pred_imds.append(img2)
        
    dice = []
    sen = []
    spe = []
    IOU = []

    for i in range(len(pred_imds)):
        dices, sens, spes, ious = evaluation(pred_imds[i], true_imds[i], verbose=0)
        dice.append(dices)
        sen.append(sens)
        spe.append(spes)
        IOU.append(ious)
        Dice = np.mean(dice)
        Sen = np.mean(sen)
        Spe = np.mean(spe)
        IoU = np.mean(IOU)

    print("============================")
    print("Dice score : {:0.4f}".format(Dice))
    print("Sensitivity score : {:0.4f}".format(Sen))
    print("Specificity score : {:0.4f}".format(Spe))
    print("IoU : {:0.4f}".format(IoU))
    print("============================")

============================Process 1/3============================
Epoch 1/10
Found 41 images belonging to 1 classes.
Found 41 images belonging to 1 classes.
500/500 [==============================] - 138s 277ms/step - loss: 0.6814 - accuracy: 0.9806

Epoch 00001: loss improved from inf to 0.68136, saving model to unet_membrane.hdf5
Epoch 2/10
500/500 [==============================] - 137s 274ms/step - loss: 0.6580 - accuracy: 0.9825

Epoch 00002: loss improved from 0.68136 to 0.65804, saving model to unet_membrane.hdf5
Epoch 3/10
500/500 [==============================] - 137s 275ms/step - loss: 0.6356 - accuracy: 0.9825

Epoch 00003: loss improved from 0.65804 to 0.63563, saving model to unet_membrane.hdf5
Epoch 4/10
500/500 [==============================] - 137s 274ms/step - loss: 0.6140 - accuracy: 0.9825

Epoch 00004: loss improved from 0.63563 to 0.61401, saving model to unet_membrane.hdf5
Epoch 5/10
500/500 [==============================] - 137s 275ms/step - loss: 0.5931 - a

In [4]:
for i in range(3):
    from model import *
    from SeLUNet_model import *
    from ResUNet_model import *
    from data import *
    import os
    import tensorflow as tf
    from skimage.io import imread
    import matplotlib.pyplot as plt
    from PIL import Image
    import cv2
    
    def evaluation(pred, true, verbose=1):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        threshold = 0.5
        for i, j in zip(pred, true):
            for x, y in zip(i, j):
                if x >= threshold and y >= threshold:
                    tp += 1
                elif x < threshold and y < threshold:
                    tn += 1
                elif x >= threshold and y < threshold:
                    fp += 1
                elif x < threshold and y >= threshold:
                    fn += 1
        sen = tp/(tp+fn)
        spe = tn/(tn+fp)
        iou = tp/(fn+fp+tp)
        dice = tp/(2*tp+fn+fp)

        if verbose==1:
            print("============================")
            print("Dice score : {:0.4f}".format(dice))
            print("Sensitivity score : {:0.4f}".format(sen))
            print("Specificity score : {:0.4f}".format(spe))
            print("IoU score : {:0.4f}".format(iou))
            print("============================")
        return dice, sen, spe, iou
    
    print('============================Process %d/3============================' % (i+1))
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    # os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')
    myGene = trainGenerator(5,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
    model = ResUNet()
    model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='dice_coef_loss',verbose=1, save_best_only=True)
    model.fit_generator(myGene,steps_per_epoch=500,epochs=10,callbacks=[model_checkpoint])

    testGene = testGenerator("data/membrane/test")
    results = model.predict_generator(testGene,15,verbose=1)
    saveResult("data/membrane/results",results)

    true_path = glob.glob("data/membrane/test_mask/*.png")
    pred_path = glob.glob("data/membrane/results/*.tif")
    true_path.sort()
    pred_path.sort()
    true_imds = []
    pred_imds = []
    for i in range(len(true_path)):
        img1 = Image.open(true_path[i])
        img1 = np.array(img1)
        img1 = img1[:,:,:3][:,:,0]
        img1 = img1 / 255
        img1[img1>0] == 1

        img2 = Image.open(pred_path[i])
        img2 = np.array(img2)
        img2 = cv2.resize(img2,(150,150))
        img2[img2>0] == 1

        true_imds.append(img1)
        pred_imds.append(img2)
        
    dice = []
    sen = []
    spe = []
    IOU = []

    for i in range(len(pred_imds)):
        dices, sens, spes, ious = evaluation(pred_imds[i], true_imds[i], verbose=0)
        dice.append(dices)
        sen.append(sens)
        spe.append(spes)
        IOU.append(ious)
        Dice = np.mean(dice)
        Sen = np.mean(sen)
        Spe = np.mean(spe)
        IoU = np.mean(IOU)

    print("============================")
    print("Dice score : {:0.4f}".format(Dice))
    print("Sensitivity score : {:0.4f}".format(Sen))
    print("Specificity score : {:0.4f}".format(Spe))
    print("IoU : {:0.4f}".format(IoU))
    print("============================")

============================Process 1/3============================


E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\ResUNet_model.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = outputs)


Epoch 1/10
Found 41 images belonging to 1 classes.
Found 41 images belonging to 1 classes.
500/500 [==============================] - 105s 210ms/step - loss: -0.5552 - dice_coef: 0.5552
Epoch 2/10


C:\Users\Guan\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with dice_coef_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


500/500 [==============================] - 95s 190ms/step - loss: -0.7333 - dice_coef: 0.7332
Epoch 3/10
500/500 [==============================] - 95s 190ms/step - loss: -0.8092 - dice_coef: 0.8092
Epoch 4/10
500/500 [==============================] - 95s 190ms/step - loss: -0.8547 - dice_coef: 0.8547
Epoch 5/10
500/500 [==============================] - 95s 190ms/step - loss: -0.8668 - dice_coef: 0.8668
Epoch 6/10
500/500 [==============================] - 95s 190ms/step - loss: -0.9037 - dice_coef: 0.9037
Epoch 7/10
500/500 [==============================] - 95s 190ms/step - loss: -0.9137 - dice_coef: 0.9137
Epoch 8/10
500/500 [==============================] - 95s 190ms/step - loss: -0.9196 - dice_coef: 0.9196
Epoch 9/10
500/500 [==============================] - 95s 190ms/step - loss: -0.8859 - dice_coef: 0.8859
Epoch 10/10
15/15 [==============================] - 1s 47ms/step


...

Epoch 1/10
Found 41 images belonging to 1 classes.
Found 41 images belonging to 1 classes.
500/500 [==============================] - 102s 203ms/step - loss: -0.5314 - dice_coef: 0.5314
Epoch 2/10


C:\Users\Guan\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with dice_coef_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


500/500 [==============================] - 96s 191ms/step - loss: -0.7159 - dice_coef: 0.7159
Epoch 3/10
500/500 [==============================] - 111s 222ms/step - loss: -0.7958 - dice_coef: 0.7958
Epoch 4/10
500/500 [==============================] - 96s 191ms/step - loss: -0.8374 - dice_coef: 0.8374
Epoch 5/10
500/500 [==============================] - 96s 192ms/step - loss: -0.8491 - dice_coef: 0.8491
Epoch 6/10
500/500 [==============================] - 96s 191ms/step - loss: -0.8806 - dice_coef: 0.8806
Epoch 7/10
500/500 [==============================] - 96s 192ms/step - loss: -0.9091 - dice_coef: 0.9091
Epoch 8/10
500/500 [==============================] - 96s 191ms/step - loss: -0.9100 - dice_coef: 0.9100
Epoch 9/10
500/500 [==============================] - 96s 191ms/step - loss: -0.9061 - dice_coef: 0.9061
Epoch 10/10
15/15 [==============================] - 1s 49ms/step


E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\1.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\3.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\4.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\5.tif is a low contrast image
  io.imsave(os.path.join(save_path, "%d.tif" % i), img_as_float(img))
E:\我的雲端硬碟\Python\untitled\medical_ai\Final_project\Plan2\unet-master-2\data.py:127: UserWarning: data/membrane/results\6.tif

Dice score : 0.0103
Sensitivity score : 0.0120
Specificity score : 0.9901
IoU : 0.0113
